In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import date
import import_ipynb
import covid_dict as cd

importing Jupyter notebook from covid_dict.ipynb


1) [Remover colunas dos atributos desconsiderados](#remove_column)  
2) [Arrumando os dados](#data_fix)  
3) [Gerando a matrix binária](#binary_matrix)  
3.1) [Matriz: Óbitos](#binary_matrix_obito)   
3.2) [Matriz: Recuperados](#binary_matrix_rec)  
3.3) [Matriz: Assintomáticos](#binary_matrix_assin)  
3.4) [Matriz: Sintomáticos](#binary_matrix_sin)  

[Panda](Panda_dataset.ipynb)

[Dicionário de dados](covid_dict.ipynb)

In [ ]:
# csv_url = "https://ti.saude.rs.gov.br/covid19/download"
#today = date.today()
# dataset = f'dataset-{today}.csv'

# req = requests.get(csv_url)
# url_content = req.content
# csv_file = open(dataset, 'wb')

# csv_file.write(url_content)
# csv_file.close()

In [2]:
df = pd.read_csv('pp_dataset/covid_rs_20210101_1.csv', sep=';')  #nrows=1000

Atributos desconsiderados:
* COD_IBGE
* MUNICIPIO
* COD_REGIAO_COVID
* DATA_SINTOMAS
* DATA_EVOLUCAO
* DATA_INCLUSAO_OBITO
* DATA_EVOLUCAO_ESTIMADA
* ETNIA_INDIGENA
* FONTE_INFORMACAO
* PAIS_NASCIMENTO
* BAIRRO

#### 1) Remover colunas dos atributos desconsiderados <a name="remove_column"></a>

In [ ]:
for e in cd.attr_removed3:
    del df[e]

In [ ]:
del df['DATA_EVOLUCAO']

#### 2) Filtrando registros -- EVOLUCAO == OBITO

In [ ]:
df.head(10)

In [ ]:
df = df[df.eq('OBITO').any(axis=1)]

#### 3) Arrumando os dados - FAIXAETARIA, DATA_CONFIRMACAO, CONDICOES <a name="data_fix"></a>

Faixa Etária
* < 1: Bebê
* (01,09): Criança
* (10, 19): Jovem
* (20, 39): Adulto 1
* (40, 49): Adulto 2
* (50, 59): Adulto 3
* (60, 69): Idoso 1
* (70, 79): Idoso 2
* (80 +): Idoso 3
       

In [ ]:
for index, row in df.iterrows():
    if row['FAIXAETARIA'] == '<1':
        row['FAIXAETARIA'] = 'Bebê'
    elif row['FAIXAETARIA'] == '01 a 04' or row['FAIXAETARIA'] == '05 a 09':
        row['FAIXAETARIA'] = 'Criança'
    elif row['FAIXAETARIA'] == '10 a 14' or row['FAIXAETARIA'] == '15 a 19':
        row['FAIXAETARIA'] = 'Jovem'
    elif row['FAIXAETARIA'] == '20 a 29' or row['FAIXAETARIA'] == '30 a 39':
        row['FAIXAETARIA'] = 'Adulto_1'
    elif row['FAIXAETARIA'] == '40 a 49':
        row['FAIXAETARIA'] = 'Adulto_2'
    elif row['FAIXAETARIA'] == '50 a 59':
        row['FAIXAETARIA'] = 'Adulto_3'
    elif row['FAIXAETARIA'] == '60 a 69':
        row['FAIXAETARIA'] = 'Idoso_1'
    elif row['FAIXAETARIA'] == '70 a 79':
        row['FAIXAETARIA'] = 'Idoso_2'
    else:
        row['FAIXAETARIA'] = 'Idoso_3'

Intervalo entre confirmação da Covid e confirmação do Óbito

In [ ]:
# df['dt_conf'] = df['DATA_CONFIRMACAO']
# df['dt_obito'] = df['DATA_INCLUSAO_OBITO']
dt_conf = pd.to_datetime(df['dt_conf'],dayfirst=True)
dt_obito = pd.to_datetime(df['dt_obito'],dayfirst=True)
df['INTERNACAO'] = round(((dt_obito - dt_conf).dt.days),-1)
del df['dt_conf']
del df['dt_obito']

Data de Confirmação Covid-19

In [ ]:
for index, row in df.iterrows():
    mes = row['DATA_CONFIRMACAO'].split('/')
    if mes[1] == '02':
        row['DATA_CONFIRMACAO'] = 'Fevereiro'
    elif mes[1] == '03':
        row['DATA_CONFIRMACAO'] = 'Março'
    elif mes[1] == '04':
        row['DATA_CONFIRMACAO'] = 'Abril'
    elif mes[1] == '05':
        row['DATA_CONFIRMACAO'] = 'Maio'    
    elif mes[1] == '06':
        row['DATA_CONFIRMACAO'] = 'Junho'
    elif mes[1] == '07':
        row['DATA_CONFIRMACAO'] = 'Julho'
    elif mes[1] == '08':
        row['DATA_CONFIRMACAO'] = 'Agosto'
    elif mes[1] == '09':
        row['DATA_CONFIRMACAO'] = 'Setembro'
    elif mes[1] == '10':
        row['DATA_CONFIRMACAO'] = 'Outubro'
    elif mes[1] == '11':
        row['DATA_CONFIRMACAO'] = 'Novembro'
    elif mes[1] == '12':
        row['DATA_CONFIRMACAO'] = 'Dezembro'
    else:
        row['DATA_CONFIRMACAO'] = 'APAGAR'

Data de confirmação óbito

In [ ]:
for index, row in df.iterrows():
    mes = row['DATA_INCLUSAO_OBITO'].split('/')
    if mes[1] == '02':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Fevereiro'
    elif mes[1] == '03':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Março'
    elif mes[1] == '04':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Abril'
    elif mes[1] == '05':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Maio'    
    elif mes[1] == '06':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Junho'
    elif mes[1] == '07':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Julho'
    elif mes[1] == '08':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Agosto'
    elif mes[1] == '09':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Setembro'
    elif mes[1] == '10':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Outubro'
    elif mes[1] == '11':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Novembro'
    elif mes[1] == '12':
        row['DATA_INCLUSAO_OBITO'] = 'Obito_Dezembro'
    else:
        row['DATA_INCLUSAO_OBITO'] = 'APAGAR'
        
        

Condições (Comorbidades)

In [ ]:
for e in cd.comorbidades:
    df[e] = str(np.nan)
    
for index, row in df.iterrows():
    if 'asma' in str(row['CONDICOES']).lower():
        row['asma'] = 'SIM'
    else:
        row['asma'] = 'NAO'
    if 'pneu' in str(row['CONDICOES']).lower():
        row['d_respiratoria'] = 'SIM'
    else:
        row['d_respiratoria'] = 'NAO'
    if 'outro' in str(row['CONDICOES']).lower():
        row['outras_comorbidades'] = 'SIM'
    else:
        row['outras_comorbidades'] = 'NAO'
    if 'neuro' in str(row['CONDICOES']).lower():
        row['d_neurologica'] = 'SIM'
    else:
        row['d_neurologica'] = 'NAO'
    if 'diabe' in str(row['CONDICOES']).lower():
        row['diabetes'] = 'SIM'
    else:
        row['diabetes'] = 'NAO'
    if 'imuno' in str(row['CONDICOES']).lower():
        row['imunodeficiencia'] = 'SIM'
    else:
        row['imunodeficiencia'] = 'NAO'
    if 'rena' in str(row['CONDICOES']).lower():
        row['d_renal'] = 'SIM'
    else:
        row['d_renal'] = 'NAO'
    if 'hepát' in str(row['CONDICOES']).lower():
        row['d_hepatica'] = 'SIM'
    else:
        row['d_hepatica'] = 'NAO'
    if 'hemato' in str(row['CONDICOES']).lower():
        row['d_hematologica'] = 'SIM'
    else:
        row['d_hematologica'] = 'NAO'
    if 'card' in str(row['CONDICOES']).lower():
        row['d_cardiaca'] = 'SIM'
    else:
        row['d_cardiaca'] = 'NAO'
    if 'obes' in str(row['CONDICOES']).lower():
        row['obesidade'] = 'SIM'
    else:
        row['obesidade'] = 'NAO'
    if 'down' in str(row['CONDICOES']).lower():
        row['sindrome_down'] = 'SIM'
    else:
        row['sindrome_down'] = 'NAO'
# del df['CONDICOES']
# del df['EVOLUCAO']

In [ ]:
# del df['CONDICOES']
# del df['EVOLUCAO']
df.head()

In [ ]:
for idx, row in df.iterrows():
    if  df.loc[idx,'CRITERIO'] == 'Clínico-Imagem' or df.loc[idx,'CRITERIO'] == 'Clínico Epidemiológico' or df.loc[idx,'CRITERIO'] == 'Outros testes' or df.loc[idx,'CRITERIO'] == 'Outros Testes' or df.loc[idx,'CRITERIO'] == 'Clínico' or df.loc[idx,'CRITERIO'] == 'Ignorado':
        df.loc[idx,'CRITERIO'] = 'Outros'

### Gerando a matrix binária <a name="binary_matrix"></a>

In [3]:
# for e in cd.municipio:
#     df[e] = df.apply(lambda row: 1 if row.MUNICIPIO == e else 0, axis=1)

for e in cd.regiao_covid:
    df[e] = df.apply(lambda row: 1 if row.REGIAO_COVID == e else 0, axis=1)
    
for e in cd.faixa_etaria2:
    df[e] = df.apply(lambda row: 1 if row.FAIXAETARIA == e else 0, axis=1)
    
for e in cd.testagem: 
    df[e] = df.apply(lambda row: 1 if row.CRITERIO == e else 0, axis=1)

for e in cd.meses:
    df[e] = df.apply(lambda row: 1 if row.DATA_CONFIRMACAO == e else 0, axis=1)
    
for e in cd.meses_obito:
    df[e] = df.apply(lambda row: 1 if row.DATA_INCLUSAO_OBITO == e else 0, axis=1)
        
for e in cd.etnia:
    df[e] = df.apply(lambda row: 1 if row.RACA_COR == e else 0, axis=1)

In [4]:
df['SexF'] = df.apply(lambda row: 1 if row.SEXO == 'Feminino' else 0, axis=1)
df['SexM'] = df.apply(lambda row: 1 if row.SEXO == 'Masculino' else 0, axis=1)
df['Hospitalizado'] = df.apply(lambda row: 1 if row.HOSPITALIZADO == 'SIM' else 0, axis=1)
df['Hosp_SRAG'] = df.apply(lambda row: 1 if row.HOSPITALIZACAO_SRAG == 'SIM' else 0, axis=1)
df['Febre'] = df.apply(lambda row: 1 if row.FEBRE == 'SIM' else 0, axis=1)
df['Tosse'] = df.apply(lambda row: 1 if row.TOSSE == 'SIM' else 0, axis=1)
df['Garganta'] = df.apply(lambda row: 1 if row.GARGANTA == 'SIM' else 0, axis=1)
df['Dispneia'] = df.apply(lambda row: 1 if row.DISPNEIA == 'SIM' else 0, axis=1)
df['Outros_Sint'] = df.apply(lambda row: 1 if row.OUTROS == 'SIM' else 0, axis=1)
df['Gestante'] = df.apply(lambda row: 1 if row.GESTANTE == 'SIM' else 0, axis=1)
df['Asma'] = df.apply(lambda row: 1 if row.asma == 'SIM' else 0, axis=1)
df['DRespiratoria'] = df.apply(lambda row: 1 if row.d_respiratoria == 'SIM' else 0, axis=1)
df['DNeurologica'] = df.apply(lambda row: 1 if row.d_neurologica == 'SIM' else 0, axis=1)
df['Diabetes'] = df.apply(lambda row: 1 if row.diabetes == 'SIM' else 0, axis=1)
df['Imunodef'] = df.apply(lambda row: 1 if row.imunodeficiencia == 'SIM' else 0, axis=1)
df['DRenal'] = df.apply(lambda row: 1 if row.d_renal == 'SIM' else 0, axis=1)
df['DHepatica'] = df.apply(lambda row: 1 if row.d_hepatica == 'SIM' else 0, axis=1)
df['DCardiaca'] = df.apply(lambda row: 1 if row.d_cardiaca == 'SIM' else 0, axis=1)
df['Obesidade'] = df.apply(lambda row: 1 if row.obesidade == 'SIM' else 0, axis=1)
df['Outras_Comorbidades'] = df.apply(lambda row: 1 if row.outras_comorbidades == 'SIM' else 0, axis=1)
df['DHematologica'] = df.apply(lambda row: 1 if row.d_hematologica == 'SIM' else 0, axis=1)
df['Sindrome_down'] = df.apply(lambda row: 1 if row.sindrome_down == 'SIM' else 0, axis=1)

In [5]:
for e in cd.old_columns3:
    del df[e]

In [7]:
del df['CONDICOES']

In [8]:
df.to_csv("pp_dataset/covid_rs_20210101_2.csv", sep=';', index=False)

In [ ]:
nome_atributos = df.columns.values.tolist()
df_dict = {i : nome_atributos[i] for i in range(0, len(nome_atributos))}

In [ ]:
df.to_csv("20201030_data_visualisation.csv", index=False)

In [ ]:
del df['d_cromossomica']

In [ ]:
df = df[df.DATA_INCLUSAO_OBITO != 'APAGAR']

In [ ]:
for i in df.columns:
    print(i)